### CleanAGROSoutput.py -- James Sayre, jsayre@ucdavis.edu



In [9]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt

### Directories
base_dir           =  "/home/j/Dropbox/"
cd_dir             =  os.path.join(base_dir, "INEGI_cds")
dosbox_dir         =  os.path.join(cd_dir,   "dosbox_output")
output_dir         =  os.path.join(cd_dir,   "Output")

### Inputs
agros_sdfs         =  os.listdir(dosbox_dir)
agros_vars         =  os.path.join(cd_dir, "AGROS_vars.csv")

### Outputs
agros_ageb         =  os.path.join(output_dir, "AGROS_ageb.csv")
ageb_ag_91         =  os.path.join(output_dir, "ageb_ag_91.csv")
mun_ag_91          =  os.path.join(output_dir, "mun_ag_91.csv")
ageb_avo_91        =  os.path.join(output_dir, "ageb_avo_91.csv")

### Programs

### Codes
state_keys = {'AGS':'01', 'BC':'02', 'BCS':'03', 'CAM':'04', 'CDMX':'09',
 'CHH':'08', 'CHP':'07', 'COA':'05', 'COL':'06', 'DUR':'10',
 'GRO':'12', 'GUA':'11', 'HDO':'13', 'JAL':'14', 'MEX':'15',
 'MIC':'16', 'MOR':'17', 'NAY':'18', 'NLE':'19', 'OAX':'20',
 'PUE':'21', 'QUE':'22', 'ROO':'23', 'SLP':'24', 'SIN':'25',
 'SON':'26', 'TAB':'27', 'TAM':'28', 'TLA':'29', 'VER':'30',
 'YUC':'31', 'ZAC':'32'}


crop_to_name_dict = {'Sesame':'Sesame seeds', 'CactusPear':'Prickly pear', 'Melon':'Melons', 'Corn':'Maize', 
                     'Chili':'Green chile', 'Magueyes':'Agave', 'SugarCane':'Sugar', 'CultivatedGrasses':'Pastures', 
                     'Avocado':'Avocados', 'Peach':'Peaches', 'Lemon':'Lemons', 'Pear':'Pears', 'Cucumber':'Cucumbers', 
                     'GreenTomatoes':'Tomatilloes', 'Sesame':'Sesame seeds','OatForage':'Oats','Peanut':'Peanuts',
                     'Onion':'Onions', 'ForageSorghum':'Sorghum silage','Zucchini':'Pumpkin','Banana':'Bananas',
                     'SweetPotato':'Sweet potato','BarleyForage':'Barley','Cauliflower':'Cauliflowers',
                     'Grape':'Grapes','CornSilage':'Maize silage','Orange':'Oranges','Estropajo':'Scourer',
                     'Romeritos':'Romerito','Strawberry':'Strawberries','Pistachio':'Pistachios',
                     'Papaya':'Papayas','Carrot':'Carrots','Cashew':'Cashews','Date':'Dates','Soy':'Soy beans',
                     'Almond':'Almonds','WheatForage':'Wheat','Apricot':'Apricots','Mandarin':'Mandarins',
                     'Cacao':'Cocoa','Coconut':'Coconuts','BroomSorghum':'Sorghum silage','BroadBean':'Green beans',
                     'Pea':'Chickpeas','Yucca':'Sweet potato','Hawthorne':'Tejocote'}

### Big warning!!! The variables differ across A and P sets in different states

In [18]:
vardf             = pd.read_csv(agros_vars)
vardf['type']     = vardf['var'].str[:1]
gvars             = ['ageb']+list(vardf[vardf['type'] == 'G']['varname'].values)+['cve_ent','ent']
avar_df           = vardf[vardf['type'] == 'A'].copy()
avar_df['states'] = avar_df['states'].apply(lambda x: x.split(','))
pvar_df           = vardf[vardf['type'] == 'P'].copy()
pvar_df['states'] = pvar_df['states'].apply(lambda x: x.split(','))

In [41]:
g_dfs, a_dfs, p_dfs = [], [], []
for sdf in agros_sdfs:
    st, datatype = sdf.split("_")
    datatype = datatype.split(".")[0]
    cve_ent = state_keys[st]
    df = pd.read_fwf(os.path.join(dosbox_dir,sdf),header=None)
    df['cve_ent'] = cve_ent
    df['ent'] = st
    if datatype == "G":
        df.columns = gvars
        g_dfs.append(df)
    elif datatype == "A":
        avars = ['ageb']+list(avar_df[avar_df['states'].apply(lambda x: st in x)]['varname'].values)+['cve_ent','ent']
        df.columns = avars
        a_dfs.append(df)
    elif datatype == "P":
        pvars = ['ageb']+list(pvar_df[pvar_df['states'].apply(lambda x: st in x)]['varname'].values)+['cve_ent','ent']
        df.columns = pvars
        p_dfs.append(df)

g_df = pd.concat(g_dfs)
a_df = pd.concat(a_dfs)
p_df = pd.concat(p_dfs)

### 2025-03-13: Some perennials were thrown into annuals, need to deal with these
false_annuals = [c for c in a_df.columns if (c in p_df.columns) and (c not in ['ageb','cve_ent','ent'])]
a_columns_renamed = []
for col in a_df.columns:
    if col in false_annuals:
        a_columns_renamed.append(col+'_2')
    else:
        a_columns_renamed.append(col)
a_df.columns = a_columns_renamed

ageb_df = g_df.merge(a_df, on=['ageb','cve_ent','ent']).merge(p_df, on=['ageb','cve_ent','ent']) ### how doesn't matter, all merges = 3
ageb_df['ageb'] = ageb_df['cve_ent'].astype(str) + ageb_df['ageb'].astype(str)
ageb_df = ageb_df[['cve_ent','ent']+[col for col in ageb_df.columns if col not in ['cve_ent','ent']]]

for fake_an in false_annuals:
    ageb_df[fake_an] = ageb_df[fake_an].fillna(ageb_df[fake_an+'_2'])
ageb_df = ageb_df.drop([c+'_2' for c in false_annuals],axis=1)

ageb_df = ageb_df.sort_values(by='ageb').reset_index(drop=True)
ageb_df.to_csv(agros_ageb, index=False)

In [43]:
ag_prod_df = ageb_df[['ageb','cve_ent','ent']+list(set(list(vardf[vardf['type'].isin(['A','P'])]['varname'].values)))]

ag_prod_df = ag_prod_df.drop(['UnsForestryArea', 'UnsAnnuCropsPV', 'TotAreaPV', 'UnsAnnuCropsOI', 'TotAreaOI', 'UnsAnnuCropsBoth', 
                              'UsedAreaBoth', 'UnsPerenCrops', 'TotAreaPeren', 'UnsGreenhouses', 'UnsNurseries', 'UnsTechAgr', 
                              'UnsDraftAnimals', 'UnsTractor', 'UnsOwnTractor', 'UnsAgrEquip', 'PineVolCut', 'OakVolCut', 
                              'OyamelVolCut', 'OtherSpeciesVol', 'UnsForestryTech', 'UnsForestProducts', 'UnsForestryEquip'], axis=1)

ag_prod_df = pd.wide_to_long(ag_prod_df, stubnames=['Uns', 'ha_sem_', 'ha_cos_', 'q_'], 
                          i=['ageb', 'cve_ent', 'ent'], j='Crop', sep='', suffix=r'\w+')

# Reset the index to make 'Crop' a column
ag_prod_df.reset_index(inplace=True)
ag_prod_df['Crop'] = ag_prod_df['Crop'].apply(lambda x: crop_to_name_dict[x] if x in crop_to_name_dict.keys() else x)
ag_prod_df['muncode'] = ag_prod_df['ageb'].str[:5]
ag_prod_df['Uns'] = ag_prod_df['Uns'].apply(lambda x: float(str(x).replace('*','1.5')))
ag_prod_df['no_data'] = (ag_prod_df['Uns'].isnull() & ag_prod_df['ha_sem_'].isnull() & ag_prod_df['ha_cos_'].isnull() & ag_prod_df['q_'].isnull())
ag_prod_df['no_data'] = ag_prod_df['no_data'].apply(lambda x: 1 if x else 0)
ag_prod_df = ag_prod_df.groupby(['ageb','cve_ent','ent','muncode','Crop']).sum().reset_index()
ag_prod_df.rename(columns={'ha_sem_':'ha_sem', 'ha_cos_':'ha_cos', 'q_':'q'}, inplace=True)
ag_prod_df.to_csv(ageb_ag_91, index=False)
### Write to muncode data
ag_prod_df.groupby(['cve_ent','ent','muncode','Crop']).sum().reset_index().drop('no_data',axis=1).to_csv(mun_ag_91, index=False)